In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision
from networks import *
from advertorch.attacks import LinfPGDAttack, GradientSignAttack, LinfBasicIterativeAttack, CarliniWagnerL2Attack, MomentumIterativeAttack, SpatialTransformAttack, LinfSPSAAttack, JacobianSaliencyMapAttack
from blackbox.CNN3 import CNN

In [2]:
num_classes=10
#torch.cuda.set_device(0)
#model = resnet(num_classes=num_classes,depth=110)
model = sixNet()
#mdoel = model.cuda()
BBox = CNN()
if True:
    model = nn.DataParallel(model,device_ids=[0,2,1,3]).cuda()
    BBox = nn.DataParallel(BBox, device_ids=[0,2,1,3]).cuda()
    
#Loading Trained Model
Bb_file = './saved_model/model_Blackbox_model_mnist'
softmax_filename= './saved_model/model_pretrain_model_mnist'
#filename= 'Models_PCL/CIFAR10_PCL.pth.tar' 
robust_model=  './saved_model/model_posttrain_model_mnist_prox'
checkpoint = torch.load(robust_model)
Bb = torch.load(Bb_file)
model.load_state_dict(checkpoint)
BBox.load_state_dict(Bb)
model.eval()
BBox.eval()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Loading Test Data (Un-normalized)
transform_test = transforms.Compose([transforms.ToTensor()])
    
testset = torchvision.datasets.MNIST(root='./file', train=False,
                                         download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, pin_memory=True,
                                        
                                          shuffle=False, num_workers=4)

eps = 0.4

In [3]:
pgd02 = LinfPGDAttack(
    BBox,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=eps,
    nb_iter=10, eps_iter=eps/10, rand_init=True, clip_min=0, clip_max=1,
    targeted=False)

fsgm = GradientSignAttack(BBox,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=eps, clip_min=0, clip_max=1)

MIM1 = MomentumIterativeAttack(BBox,loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=eps, nb_iter=10, eps_iter=eps/10, clip_min=0, clip_max=1, targeted=False)
BIM = LinfBasicIterativeAttack(BBox, loss_fn=nn.CrossEntropyLoss(reduction="sum"),eps=eps, nb_iter=10, eps_iter=eps/10, clip_min=0, clip_max=1, targeted=False)
CW = CarliniWagnerL2Attack(BBox,loss_fn=nn.CrossEntropyLoss(reduction="sum"), num_classes=10, learning_rate=0.01, max_iterations=1000, initial_const=10, clip_min=0, clip_max=1)
attacks={'pgd':pgd02,
        "mim":MIM1,
        "fsgm":fsgm,
        "BIM":BIM,
        "CW":CW}

/home/C00289092/anaconda3/envs/advattack/lib/python3.8/site-packages/advertorch/attacks/carlini_wagner.py:68: UserWarning: This Attack currently do not support a different loss function other than the default. Setting loss_fn manually is not effective.
  warnings.warn(


In [ ]:
model.eval()
for attack_name in attacks:
    correct = 0
    size = 0 
    n = 0
    for tedata, tetarget in test_loader:
        n += 1
        size += tedata.shape[0]
        tedata, tetarget = tedata.to(device), tetarget.to(device)

        tedata = attacks[attack_name].perturb(tedata, tetarget).to(device)
        output = model(tedata)
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(tetarget.view_as(pred)).sum().item()

    print("{:s} acc: {:.2f}".format(attack_name, 100. * correct / size))

pgd acc: 43.13
mim acc: 26.99
fsgm acc: 47.27
BIM acc: 46.20
